In [0]:
"""
https://delta.io/blog/remove-files-delta-lake-vacuum-command/
https://github.com/delta-io/delta-examples/blob/master/notebooks/pyspark/vacuum.ipynb
"""

"""
Delta tables can be saved in a variety of storage systems like HDFS, AWS S3, or Azure Blob Storage. These storage systems usually cost money.

If your Delta table contains a lot of tombstoned files that you don't need because you're not time traveling beyond the retention period, then you can just vacuum the tombstoned files and save on storage costs.

Vacuuming obviously isn't viable in all situations. You may need to keep tombstoned files so you can time travel, for regulatory requirements, or for audit data retention. Vacuuming is a great way to save on storage costs, but it's not always desirable.
"""

# Delta Lake vacuum example


In [0]:
df = spark.createDataFrame([("bob", 3), ("sue", 5)]).toDF("first_name", "age")
df.repartition(1).write.format("delta").saveAsTable("vacuum_example")

In [0]:
df = spark.createDataFrame([("ingrid", 58), ("luisa", 87)]).toDF("first_name", "age")
df.repartition(1).write.format("delta").mode("append").saveAsTable("vacuum_example")

In [0]:
spark.table("vacuum_example").show()

+----------+---+
|first_name|age|
+----------+---+
|    ingrid| 58|
|     luisa| 87|
|       bob|  3|
|       sue|  5|
+----------+---+



# First pass at vacuum

In [0]:
spark.sql("VACUUM vacuum_example DRY RUN")

Out[10]: DataFrame[path: string]

# Overwrite Delta table to tombstone files


In [0]:
df = spark.createDataFrame([("jordana", 26), ("fred", 25)]).toDF("first_name", "age")
df.repartition(1).write.format("delta").mode("overwrite").saveAsTable("vacuum_example")
spark.table("vacuum_example").show()


+----------+---+
|first_name|age|
+----------+---+
|   jordana| 26|
|      fred| 25|
+----------+---+



In [0]:
spark.sql("VACUUM vacuum_example DRY RUN") # JDBC prj had 7 days of vacuum (24*7=168hr)

Out[12]: DataFrame[path: string]

In [0]:
spark.sql("VACUUM vacuum_example RETAIN 0 HOURS DRY RUN")

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-2218377028571736>:1
----> 1 spark.sql("VACUUM some_people RETAIN 0 HOURS DRY RUN")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (args or {}).items()}
-> 1387     return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
   1388 finally:
   1389     if len(kwargs) > 0:

File /databricks/spark/python/lib/py4j-0


# DISABLE RETENTION DURATION CHECK (DON'T DO THIS IN PROD)


In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [0]:
spark.sql("VACUUM vacuum_example RETAIN 0 HOURS DRY RUN").show(truncate=False)

+------------------------------------------------------------------------------------------------------------+
|path                                                                                                        |
+------------------------------------------------------------------------------------------------------------+
|dbfs:/user/hive/warehouse/vacuum_example/part-00000-4bd594ac-6716-4ede-9801-5bc1783f3ce1-c000.snappy.parquet|
|dbfs:/user/hive/warehouse/vacuum_example/part-00000-93326e8e-b633-4016-9c0e-82107ec35d96-c000.snappy.parquet|
+------------------------------------------------------------------------------------------------------------+



In [0]:
spark.sql("VACUUM vacuum_example RETAIN 0 HOURS")

Out[17]: DataFrame[path: string]

# Vacuuming limits ability to time travel


In [0]:
spark.sql("SELECT * FROM vacuum_example").show()

+----------+---+
|first_name|age|
+----------+---+
|   jordana| 26|
|      fred| 25|
+----------+---+



In [0]:
spark.sql("SELECT * FROM vacuum_example VERSION AS OF 2").show()

+----------+---+
|first_name|age|
+----------+---+
|   jordana| 26|
|      fred| 25|
+----------+---+



In [0]:
spark.sql("SELECT * FROM vacuum_example VERSION AS OF 1").show()
"""
versionAsOf 1 is physically removed by vacuum command, can't time travel
"""

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2218377028571744>:1
----> 1 spark.sql("SELECT * FROM vacuum_example VERSION AS OF 1").show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         message_parameters={"arg_name": "vertical", "arg_type": type(vertical).__name__},
    917     )
    919 if isinstance(truncate, bool) and truncate:
--> 920     pri